## 1. Download and Explore Dataset

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
df = pd.read_csv('Toronto2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3L,North York,Downsview West,43.739015,-79.506944
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
3,M8V,Etobicoke,New Toronto,43.605647,-79.501321
4,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [4]:
def check(Borough):
    if 'Toronto' in Borough.split():
        return True
    else:
        return False

In [6]:
df_Toronto = df[[check(x) for x in df['Borough']]]
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259
7,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
13,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(set(df_Toronto['Borough'])), len(set(df_Toronto['Neighborhood']))
    )
)

The dataframe has 4 boroughs and 27 neighborhoods.


In [8]:
# !conda install -c conda-forge geopy --yes 

In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup='blue',
        fill=True,
        fill_color='3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)
map_toronto

#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'O3VIGWURXLK1A410UCKYHTVDI1422OW0Y2MKIH4BODO2UPXX' # your Foursquare ID
CLIENT_SECRET = 'JSTLAWZECUSUMGBW3VYTLK5DB2MFKTWGMFIPBV1E02CZZXHX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O3VIGWURXLK1A410UCKYHTVDI1422OW0Y2MKIH4BODO2UPXX
CLIENT_SECRET:JSTLAWZECUSUMGBW3VYTLK5DB2MFKTWGMFIPBV1E02CZZXHX


In [14]:
df_Toronto.loc[1, 'Neighborhood'].split(',')[0]

'Little Portugal'

In [15]:
n_lat = df_Toronto.loc[1, 'Latitude']
n_lon = df_Toronto.loc[1, 'Longitude']
n_name = df_Toronto.loc[1, 'Neighborhood'].split(',')[0]
print('Latitude and longitude values of {} are {}, {}.'.format(n_name, n_lat, n_lon))

Latitude and longitude values of Little Portugal are 43.647926700000006, -79.4197497.


In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_lat, 
    n_lon, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=O3VIGWURXLK1A410UCKYHTVDI1422OW0Y2MKIH4BODO2UPXX&client_secret=JSTLAWZECUSUMGBW3VYTLK5DB2MFKTWGMFIPBV1E02CZZXHX&v=20180605&ll=43.647926700000006,-79.4197497&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5641728fc77c0023989a94'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Trinity Bellwoods',
  'headerFullLocation': 'Trinity Bellwoods, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 65,
  'suggestedBounds': {'ne': {'lat': 43.65242670450001,
    'lng': -79.41354235474367},
   'sw': {'lat': 43.643426695500004, 'lng': -79.42595704525633}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f7891c7e4b0b9643b73e08d',
       'name': 'Bellwoods Brewery',
       'location': {'address': '124 Ossington Ave',
        'crossStreet': 'at Argyle St',
        'lat': 43.647097254598236,
        'lng': -79.41995537873463,
        '

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Bellwoods Brewery,Brewery,43.647097,-79.419955
1,Pizzeria Libretto,Pizza Place,43.648979,-79.420604
2,OddSeoul,Korean Restaurant,43.646192,-79.419601
3,Foxley Bistro,Asian Restaurant,43.648643,-79.420495
4,Bang Bang Ice Cream & Bakery,Ice Cream Shop,43.646246,-79.419553


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

65 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                latitudes=df_Toronto['Latitude'],
                                longitudes=df_Toronto['Longitude'])

Little Portugal, Trinity
Dovercourt Village
Design Exchange, Toronto Dominion Centre
Church and Wellesley
The Annex, Yorkville
Rosedale
High Park
Moore Park
Berczy Park
Deer Park, Rathnelly, South Hill
Chinatown, Grange Park, Kensington Market
Forest Hill North
Parkdale, Roncesvalles
Runnymede, Swansea
Cabbagetown, St. James Town
India Bazaar
St. James Town
University of Toronto
Exhibition Place, Parkdale Village
Lawrence Park
Harbourfront, Regent Park
Commerce Court
CN Tower, King and Spadina, Railway Lands, South Niagara
Toronto Islands, Union Station
Riverdale
First Canadian Place, Underground city
The Beaches


In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(1175, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Little Portugal, Trinity",43.647927,-79.41975,Bellwoods Brewery,43.647097,-79.419955,Brewery
1,"Little Portugal, Trinity",43.647927,-79.41975,Pizzeria Libretto,43.648979,-79.420604,Pizza Place
2,"Little Portugal, Trinity",43.647927,-79.41975,OddSeoul,43.646192,-79.419601,Korean Restaurant
3,"Little Portugal, Trinity",43.647927,-79.41975,Foxley Bistro,43.648643,-79.420495,Asian Restaurant
4,"Little Portugal, Trinity",43.647927,-79.41975,Bang Bang Ice Cream & Bakery,43.646246,-79.419553,Ice Cream Shop


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"CN Tower, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",45,45,45,45,45,45
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Church and Wellesley,84,84,84,84,84,84
Commerce Court,100,100,100,100,100,100
"Deer Park, Rathnelly, South Hill",16,16,16,16,16,16
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
Dovercourt Village,14,14,14,14,14,14


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


## 3. Analyze Each Neighborhood

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Little Portugal, Trinity",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Little Portugal, Trinity",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
toronto_onehot.shape

(1175, 210)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.017857,0.017857,0.035714,0.000000,0.000000,0.00,0.017857,0.017857,0.035714,0.000000,0.017857,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.017857,0.035714,0.000000,0.00,0.000000,0.035714,0.000000,0.000000,0.00,0.000000,0.00,0.053571,0.089286,0.000000,0.000000,0.00,0.017857,0.017857,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.00,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.017857,0.017857,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.017857,0.017857,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.017857,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.017857,0.017857,0.000000,0.01785

#### Let's confirm the new size

In [31]:
toronto_grouped.shape

(27, 210)

#### Let's print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3              Bakery  0.04
4  Seafood Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4          Boutique  0.07


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.04
2              Bakery  0.04
3                Café  0.04
4          Restaurant  0.04


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0                           Café  0.07
1  Vegetarian / Vegan Restaurant  0.06
2             Chinese Restaurant  0.05
3                            Bar  0.04
4          Vietnamese Restaurant  0.04


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07
1  Japan

#### Let's put that into a *pandas* dataframe

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Steakhouse,Farmers Market,Cheese Shop,Café,Beer Bar,Park
1,"CN Tower, King and Spadina, Railway Lands, Sou...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Coffee Shop,Boat or Ferry,Boutique,Bar,Airport Food Court,Airport
2,"Cabbagetown, St. James Town",Coffee Shop,Pub,Restaurant,Café,Bakery,Pizza Place,Italian Restaurant,Park,Playground,Beer Store
3,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Bar,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Park
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Mediterranean Restaurant,Men's Store,Café,Gastropub


## 4. Cluster Neighborhoods

In [36]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,1,Bar,Coffee Shop,Asian Restaurant,Wine Bar,Cocktail Bar,Pizza Place,New American Restaurant,Men's Store,Vietnamese Restaurant,French Restaurant
2,M6H,West Toronto,Dovercourt Village,43.669005,-79.442259,1,Supermarket,Bakery,Pharmacy,Park,Music Venue,Bank,Brewery,Middle Eastern Restaurant,Café,Gym / Fitness Center
7,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,1,Coffee Shop,Café,Hotel,Restaurant,Bar,Deli / Bodega,American Restaurant,Italian Restaurant,Bakery,Gym
8,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Mediterranean Restaurant,Men's Store,Café,Gastropub
13,M5R,Central Toronto,"The Annex, Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,Pizza Place,Cosmetics Shop,Flower Shop,Indian Restaurant,Pub,Burger Joint,BBQ Joint


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,0,Restaurant,Playground,Tennis Court,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 2

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,West Toronto,1,Bar,Coffee Shop,Asian Restaurant,Wine Bar,Cocktail Bar,Pizza Place,New American Restaurant,Men's Store,Vietnamese Restaurant,French Restaurant
2,West Toronto,1,Supermarket,Bakery,Pharmacy,Park,Music Venue,Bank,Brewery,Middle Eastern Restaurant,Café,Gym / Fitness Center
7,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Bar,Deli / Bodega,American Restaurant,Italian Restaurant,Bakery,Gym
8,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Mediterranean Restaurant,Men's Store,Café,Gastropub
13,Central Toronto,1,Sandwich Place,Café,Coffee Shop,Pizza Place,Cosmetics Shop,Flower Shop,Indian Restaurant,Pub,Burger Joint,BBQ Joint
21,West Toronto,1,Mexican Restaurant,Café,Grocery Store,Fried Chicken Joint,Park,Music Venue,Diner,Discount Store,Cajun / Creole Restaurant,Fast Food Restaurant
26,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Steakhouse,Farmers Market,Cheese Shop,Café,Beer Bar,Park
28,Central Toronto,1,Coffee Shop,Light Rail Station,Pub,Bagel Shop,Vietnamese Restaurant,Supermarket,Sushi Restaurant,American Restaurant,Pizza Place,Liquor Store
30,Downtown Toronto,1,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Bar,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Park
35,West Toronto,1,Breakfast Spot,Gift Shop,Restaurant,Italian Restaurant,Bank,Dog Run,Eastern European Restaurant,Movie Theater,Dessert Shop,Coffee Shop


#### Cluster 3

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Central Toronto,2,Park,Bus Line,Swim School,Wine Bar,Dive Bar,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 4

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Park,Playground,Trail,Building,Discount Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5 

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,4,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Wine Bar,Dog Run,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
